<a href="https://colab.research.google.com/github/jaozomar/horseRaceDetection/blob/main/FinalProjectECS174.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# if using weights file from drive
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!git clone https://github.com/akTwelve/Mask_RCNN.git

In [ ]:
%cd Mask_RCNN
!echo -e "\npixellib" >> requirements.txt
!pip3 install -r requirements.txt
%cd ..

/usr/local/lib/python3.10/dist-packages/pixellib/instance/utils.py

line 566/569: change `np.bool` to `bool`

/content/Mask_RCNN/mrcnn/utils.py

line 533/551/571/574: change `np.bool` to `bool`

line 901/907: change `order=order` to `order=0`

/usr/local/lib/python3.10/dist-packages/pixellib/semantic/deeplab.py

line 15: change `tensorflow.python.keras.layers` to `tensorflow.keras.layers`

/content/Mask_RCNN/mrcnn/model.py

line 1273: change `np.bool` to `bool`

line 2141: change `from keras.utils.data_utils` to `from tensorflow.keras.utils`

line 2355: change `multiprocessing.cpu_count()` to `1`

restart session

In [ ]:
%cd Mask_RCNN
!python3 setup.py install
%cd ..

In [ ]:
!git clone https://github.com/waleedka/coco.git
%cd coco/PythonAPI
!make
%cd ../..

In [ ]:
import os
import time
import numpy as np
import imgaug
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils

import zipfile
import urllib.request
import shutil

# Import Mask RCNN
import sys
sys.path.insert(0, '/content/Mask_RCNN')
from mrcnn.config import Config
from mrcnn import model as modellib, utils

horse_class_id = 18

############################################################
#  Configurations
############################################################

class CocoConfig(Config):
    """Configuration for training on MS COCO.
    Derives from the base Config class and overrides values specific
    to the COCO dataset.
    """
    # Give the configuration a recognizable name
    NAME = "coco"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Uncomment to train on 8 GPUs (default is 1)
    # GPU_COUNT = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 80  # COCO has 80 classes

class InferenceConfig(CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0

############################################################
#  Dataset
############################################################

class CocoDataset(utils.Dataset):
    def load_coco(self, dataset_dir, subset, year, class_ids=None,
                  class_map=None, return_coco=False, auto_download=False):
        """Load a subset of the COCO dataset.
        dataset_dir: The root directory of the COCO dataset.
        subset: What to load (train, val, minival, valminusminival)
        year: What dataset year to load (2014, 2017) as a string, not an integer
        class_ids: If provided, only loads images that have the given classes.
        class_map: TODO: Not implemented yet. Supports maping classes from
            different datasets to the same class ID.
        return_coco: If True, returns the COCO object.
        auto_download: Automatically download and unzip MS-COCO images and annotations
        """

        if auto_download is True:
            self.auto_download(dataset_dir, subset, year)

        coco = COCO("{}/annotations/instances_{}{}.json".format(dataset_dir, subset, year))
        if subset == "minival" or subset == "valminusminival":
            subset = "val"
        image_dir = "{}/{}{}".format(dataset_dir, subset, year)

        # Load all classes or a subset?
        if not class_ids:
            # All classes
            class_ids = sorted(coco.getCatIds())

        # All images or a subset?
        if class_ids:
            image_ids = []
            for id in class_ids:
                image_ids.extend(list(coco.getImgIds(catIds=[id])))
            # Remove duplicates
            image_ids = list(set(image_ids))
        else:
            # All images
            image_ids = list(coco.imgs.keys())

        # Add classes
        for i in class_ids:
            self.add_class("coco", i, coco.loadCats(i)[0]["name"])

        # Add images
        for i in image_ids:
            annotations = coco.loadAnns(coco.getAnnIds(imgIds=[i], catIds=class_ids, iscrowd=None))
            if len(annotations) > 0:  # Only add images that contain the specified class
                self.add_image(
                    "coco", image_id=i,
                    path=os.path.join(image_dir, coco.imgs[i]['file_name']),
                    width=coco.imgs[i]["width"],
                    height=coco.imgs[i]["height"],
                    annotations=annotations
                )
        if return_coco:
            return coco

    def auto_download(self, dataDir, dataType, dataYear):
        """Download the COCO dataset/annotations if requested.
        dataDir: The root directory of the COCO dataset.
        dataType: What to load (train, val, minival, valminusminival)
        dataYear: What dataset year to load (2014, 2017) as a string, not an integer
        Note:
            For 2014, use "train", "val", "minival", or "valminusminival"
            For 2017, only "train" and "val" annotations are available
        """

        # Setup paths and file names
        if dataType == "minival" or dataType == "valminusminival":
            imgDir = "{}/{}{}".format(dataDir, "val", dataYear)
            imgZipFile = "{}/{}{}.zip".format(dataDir, "val", dataYear)
            imgURL = "http://images.cocodataset.org/zips/{}{}.zip".format("val", dataYear)
        else:
            imgDir = "{}/{}{}".format(dataDir, dataType, dataYear)
            imgZipFile = "{}/{}{}.zip".format(dataDir, dataType, dataYear)
            imgURL = "http://images.cocodataset.org/zips/{}{}.zip".format(dataType, dataYear)
        # print("Image paths:"); print(imgDir); print(imgZipFile); print(imgURL)

        # Create main folder if it doesn't exist yet
        if not os.path.exists(dataDir):
            os.makedirs(dataDir)

        # Download images if not available locally
        if not os.path.exists(imgDir):
            os.makedirs(imgDir)
            print("Downloading images to " + imgZipFile + " ...")
            with urllib.request.urlopen(imgURL) as resp, open(imgZipFile, 'wb') as out:
                shutil.copyfileobj(resp, out)
            print("... done downloading.")
            print("Unzipping " + imgZipFile)
            with zipfile.ZipFile(imgZipFile, "r") as zip_ref:
                zip_ref.extractall(dataDir)
            print("... done unzipping")
        print("Will use images in " + imgDir)

        # Setup annotations data paths
        annDir = "{}/annotations".format(dataDir)
        if dataType == "minival":
            annZipFile = "{}/instances_minival2014.json.zip".format(dataDir)
            annFile = "{}/instances_minival2014.json".format(annDir)
            annURL = "https://dl.dropboxusercontent.com/s/o43o90bna78omob/instances_minival2014.json.zip?dl=0"
            unZipDir = annDir
        elif dataType == "valminusminival":
            annZipFile = "{}/instances_valminusminival2014.json.zip".format(dataDir)
            annFile = "{}/instances_valminusminival2014.json".format(annDir)
            annURL = "https://dl.dropboxusercontent.com/s/s3tw5zcg7395368/instances_valminusminival2014.json.zip?dl=0"
            unZipDir = annDir
        else:
            annZipFile = "{}/annotations_trainval{}.zip".format(dataDir, dataYear)
            annFile = "{}/instances_{}{}.json".format(annDir, dataType, dataYear)
            annURL = "http://images.cocodataset.org/annotations/annotations_trainval{}.zip".format(dataYear)
            unZipDir = dataDir
        # print("Annotations paths:"); print(annDir); print(annFile); print(annZipFile); print(annURL)

        # Download annotations if not available locally
        if not os.path.exists(annDir):
            os.makedirs(annDir)
        if not os.path.exists(annFile):
            if not os.path.exists(annZipFile):
                print("Downloading zipped annotations to " + annZipFile + " ...")
                with urllib.request.urlopen(annURL) as resp, open(annZipFile, 'wb') as out:
                    shutil.copyfileobj(resp, out)
                print("... done downloading.")
            print("Unzipping " + annZipFile)
            with zipfile.ZipFile(annZipFile, "r") as zip_ref:
                zip_ref.extractall(unZipDir)
            print("... done unzipping")
        print("Will use annotations in " + annFile)

    def load_mask(self, image_id):
        """Load instance masks for the given image.

        Different datasets use different ways to store masks. This
        function converts the different mask format to one format
        in the form of a bitmap [height, width, instances].

        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a COCO image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "coco":
            return super(CocoDataset, self).load_mask(image_id)

        instance_masks = []
        class_ids = []
        annotations = self.image_info[image_id]["annotations"]
        # Build mask of shape [height, width, instance_count] and list
        # of class IDs that correspond to each channel of the mask.
        for annotation in annotations:
            class_id = self.map_source_class_id(
                "coco.{}".format(annotation['category_id']))
            if class_id:
                m = self.annToMask(annotation, image_info["height"],
                                   image_info["width"])
                # Some objects are so small that they're less than 1 pixel area
                # and end up rounded out. Skip those objects.
                if m.max() < 1:
                    continue
                # Is it a crowd? If so, use a negative class ID.
                if annotation['iscrowd']:
                    # Use negative class ID for crowds
                    class_id *= -1
                    # For crowd masks, annToMask() sometimes returns a mask
                    # smaller than the given dimensions. If so, resize it.
                    if m.shape[0] != image_info["height"] or m.shape[1] != image_info["width"]:
                        m = np.ones([image_info["height"], image_info["width"]], dtype=bool)
                instance_masks.append(m)
                class_ids.append(class_id)

        # Pack instance masks into an array
        if class_ids:
            mask = np.stack(instance_masks, axis=2).astype(bool)
            class_ids = np.array(class_ids, dtype=np.int32)
            return mask, class_ids
        else:
            # Call super class to return an empty mask
            return super(CocoDataset, self).load_mask(image_id)

    def image_reference(self, image_id):
        """Return a link to the image in the COCO Website."""
        info = self.image_info[image_id]
        if info["source"] == "coco":
            return "http://cocodataset.org/#explore?id={}".format(info["id"])
        else:
            super(CocoDataset, self).image_reference(image_id)

    # The following two functions are from pycocotools with a few changes.

    def annToRLE(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE to RLE.
        :return: binary mask (numpy 2D array)
        """
        segm = ann['segmentation']
        if isinstance(segm, list):
            # polygon -- a single object might consist of multiple parts
            # we merge all parts into one mask rle code
            rles = maskUtils.frPyObjects(segm, height, width)
            rle = maskUtils.merge(rles)
        elif isinstance(segm['counts'], list):
            # uncompressed RLE
            rle = maskUtils.frPyObjects(segm, height, width)
        else:
            # rle
            rle = ann['segmentation']
        return rle

    def annToMask(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE, or RLE to binary mask.
        :return: binary mask (numpy 2D array)
        """
        rle = self.annToRLE(ann, height, width)
        m = maskUtils.decode(rle)
        return m

In [ ]:
############################################################
#  Train
############################################################

# Configurations
config = CocoConfig()
# config.display()

# Create model
model = modellib.MaskRCNN(mode="training", config=config, model_dir="./logs")

# Load weights
model_path = model.get_imagenet_weights()
print("Loading weights ", model_path)
model.load_weights(model_path, by_name=True)

dataset_train = CocoDataset()
dataset_train.load_coco("./dataset", "train", year="2017", class_ids=[horse_class_id], auto_download=True)
dataset_train.prepare()

dataset_val = CocoDataset()
dataset_val.load_coco("./dataset", "val", year="2017", class_ids=[horse_class_id], auto_download=True)
dataset_val.prepare()

# Image Augmentation
# Right/Left flip 50% of the time
augmentation = imgaug.augmenters.Fliplr(0.5)

In [ ]:
print("Training")
model.train(dataset_train, dataset_val, learning_rate=config.LEARNING_RATE,
                    epochs=20, layers='heads', augmentation=augmentation)

In [ ]:
MODEL_PATH = "./final_weights.h5"

In [ ]:
############################################################
#  Apply
############################################################

import pixellib
from pixellib.semantic import semantic_segmentation
from pixellib.instance import instance_segmentation

def pixellibCoco(pictureFile):
    # Open a segment model of Mask R_CNN model trained on Microsoft Coco dataset
    segment_image_ist = instance_segmentation()
    segment_image_ist.load_model(MODEL_PATH)
    results, segoverlay = segment_image_ist.segmentImage(pictureFile, output_image_name = "instance_"+pictureFile, show_bboxes = True)
    return results, segoverlay

def findWinner(results, direction):
    rois = results['rois']
    class_ids = results['class_ids']
    masks = results['masks']

    winning_horse_i = -1
    if direction == 'left':
        winning_horse_x = 1000
    else:
        winning_horse_x = -1000

    for i, class_id in enumerate(class_ids):
        if class_id == horse_class_id:
            if direction == 'left':
                curr_x = rois[i][1]
            else:
                curr_x = rois[i][3]
            if direction == 'left' and curr_x < winning_horse_x or direction == 'right' and curr_x > winning_horse_x:
                winning_horse_x = curr_x
                winning_horse_i = i
    if winning_horse_i != -1:
        return masks[:, :, winning_horse_i], winning_horse_x
    print("no horse")

def overlayMask(image, mask):
    color = [255, 215, 0]  # Gold
    color_mask = np.zeros_like(image)
    for i in range(3):
        color_mask[:, :, i] = mask * color[i]
    return cv2.addWeighted(image, 1.0, color_mask, 0.5, 0)

In [ ]:
# pictureFile = "01012018R3C2.png" # left
# pictureFile = "01012018R3C5.png" # left
# pictureFile = "01012018R3C7.png" # left
# pictureFile = "01012019R1C1.png" # right
# pictureFile = "01022019R2C5.png" # left
pictureFile = "01022021R4C3.png" # left
# pictureFile = "01012019R3C7.png" # right
# pictureFile = "01012020R1C8.png" # right
race_direction = 'left'

In [ ]:
results, segoverlay = pixellibCoco(pictureFile)
mask, winner_x = findWinner(results, race_direction)

In [ ]:
import cv2
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(5, 8))

img = cv2.imread(pictureFile, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fig.add_subplot(3, 1, 1)
plt.imshow(img)
plt.axis('off')
plt.title("Original Picture: "+pictureFile)

segoverlay = cv2.cvtColor(segoverlay, cv2.COLOR_BGR2RGB)
fig.add_subplot(3, 1, 2)
plt.imshow(segoverlay)
plt.axis('off')
plt.title("Segmentation Overlay")

winoverlay = overlayMask(img, mask)
fig.add_subplot(3, 1, 3)
plt.imshow(winoverlay)
plt.axvline(x=winner_x, color='red', linewidth=2)
plt.axis('off')
plt.title("Winner Overlay")

plt.show()